In [3]:
%config IPCompleter.greedy=True

In [29]:
import argparse
import sys

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
if __name__ == '__main__':
    # Argument parser to parse command line arguments
    parser = argparse.ArgumentParser()

    parser.add_argument('--dataset', dest='dataset_path', action='store', type=str, help='path to dataset')
    parser.add_argument('--mode', dest='mode', action='store', type=str, help='mode of algorithm', default='erm')

#     args = parser.parse_args('--dataset /home/rmodi/Documents/20_Spring/CSE512/hw1/Breast_cancer_data.csv --mode erm'.split())
#     args = parser.parse_args('--dataset /home/rmodi/Documents/20_Spring/CSE512/hw1/Breast_cancer_data.csv --mode cfv'.split())
    args = parser.parse_args('--dataset /home/rmodi/Documents/20_Spring/CSE512/hw1/linearly-separable-dataset.csv --mode erm'.split())
#     args = parser.parse_args('--dataset /home/rmodi/Documents/20_Spring/CSE512/hw1/linearly-separable-dataset.csv --mode cfv'.split())

    # Get the filenames for the training and testing data along with the filenames of their target values data
    train_file, test_file = str(args.train_dataset), str(args.test_dataset)
    y_train_file, y_test_file = train_file.replace('.csv', '_label.csv'), test_file.replace('.csv', '_label.csv')
    # Get the dataframes for training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_file, test_file, y_train_file, y_test_file)

    # Create a ID3DecisionTree model and fit the model for the given training data
    model = ID3DecisionTree(args.p_value_threshold).fit(X_train, y_train)
    model.save(args.decision_tree)  # Save the model at the specified location

    preds = model.predict(X_test)  # Get the predictions for the test dataset
    preds.to_csv(args.output_file, header=False, index=False)  # Write the predictions to the output file

    print('Number of internal nodes: {} \nNumber of leaf nodes: {}'.format(model.internal_nodes_count,
                                                                           model.leaf_nodes_count))
    if y_test is not None:
        accuracy = accuracy_score(y_test, preds)  # Calculate accuracy if the target values for test data is given
        print('Model Accuracy: {}'.format(accuracy))


AttributeError: 'Namespace' object has no attribute 'train_dataset'

In [6]:
df = pd.read_csv(args.dataset_path)

In [7]:
df.head()

,x1,x2,y
0,0.876607,3.007882,1
1,0.229373,0.661393,0
2,0.241416,1.610415,1
3,0.831164,1.879528,0
4,0.309244,0.663247,0


In [8]:
y = df.iloc[:, -1].values
y = np.where(y == 0, -1, 1)

In [9]:
y.shape

(1151,)

In [10]:
X = df.iloc[:, :-1].values

In [11]:
X.shape

(1151, 2)

In [12]:
X

array([[0.8766075 , 3.00788153],
       [0.22937269, 0.66139291],
       [0.24141585, 1.61041534],
       ...,
       [0.11625614, 0.94551889],
       [0.74495011, 1.12962314],
       [0.22731161, 1.41663829]])

In [13]:
m, d = X.shape

In [14]:
D = np.array([1/m for i in range(m)])

In [15]:
D.shape

(1151,)

In [16]:
m, d = X.shape
F_star, theta_star, j_star = float('inf'), 0, 0
for j in range(d):
    xj = np.sort(X[:, j])
    xj = np.append(xj, xj[-1] + 1)
    F = np.sum(np.where(y == 1, 1, 0) * D)
    if F < F_star:
        F_star, theta_star, j_star = F, xj[0] - 1, j
    for i in range(m):
        F = F - y[i]*D[i]
        if F < F_star and i < (m-1) and xj[i] != xj[i+1]:
            F_star, theta_star, j_star = F, (xj[i] + xj[i+1])/2, j
print(theta_star, j_star)

0.04088599619575878 0


In [17]:
def weak_learner(X, y, D):
    m, d = X.shape
    F_star, theta_star, j_star = float('inf'), 0, 0
    for j in range(d):
        xj = np.sort(X[:, j])
        xj = np.append(xj, xj[-1] + 1)
        F = np.sum(np.where(y == 1, 1, 0) * D)
        if F < F_star:
            F_star, theta_star, j_star = F, xj[0] - 1, j
        for i in range(m):
            F = F - y[i]*D[i]
            if F < F_star and i < (m-1) and xj[i] != xj[i+1]:
                F_star, theta_star, j_star = F, (xj[i] + xj[i+1])/2, j
    return theta_star, j_star

In [64]:
D = np.array([1/m for i in range(m)])
T = 10
weak_learners, errors, weights = [], [], []

for t in range(T):
    weak_learners.append(weak_learner(X, y, D))
    theta, j = weak_learners[-1]
    preds = np.where((theta - X[:, j]) <= 0, 1, -1)
    error = np.sum(D[np.where(preds != y)])
    errors.append(error)
    w = 0.5 * math.log(1/error - 1)
    weights.append(w)
    D = D * np.exp(-w * y * preds)
    D = D / sum(D)

In [65]:
weights

[-0.01824702921528571,
 -6.342686284972269e-05,
 -2.2648549702353708e-14,
 5.773159728050781e-15,
 4.662936703425635e-15,
 1.7763568394002473e-15,
 1.7763568394002473e-15,
 1.5543122344752168e-15,
 -3.197442310920553e-14,
 1.5987211554602e-14]

In [71]:
preds = np.zeros(y.shape)
for i, learner in enumerate(weak_learners):
    theta, j = learner
    preds = preds + weights[i] * np.where((theta - X[:, j]) <= 0, 1, -1)
preds = np.where(preds <= 0, 1, -1)

In [76]:
len(np.where(preds != y)[0])/m

0.5091225021720244

In [78]:
y.shape

(1151,)

In [40]:
k = 10

In [41]:
s = int(m/k)

In [49]:
batches = []
for i in range(k):
    start_index, end_index = s*i, s*(i+1)
    if i < (k-1):
        batches.append((X[start_index:end_index], y[start_index:end_index]))
    else:
        batches.append((X[start_index:], y[start_index:]))

In [129]:
weights, errors = [], []
for i in range(k):
    train_X, train_y, test_X, test_y = None, None, None, None
    for j, (X, y) in enumerate(batches):
        if j == i:
            test_X, test_y = X, y
        else:
            if train_X is None:
                train_X, train_y = X, y
            else:
                train_X, train_y = np.append(train_X, X, axis=0), np.append(train_y, y)
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    weights.append(train(train_X, train_y))
    errors.append(error(test_X, test_y, weights[-1]))

(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(513, 6) (513,) (56, 6) (56,)
(513, 6) (513,)
(504, 6) (504,) (65, 6) (65,)
(504, 6) (504,)


In [131]:
weights

[array([ 1602637.45498681, -1247268.25006665,  1628099.43005266,
         -204412.19999828,   -59284.70236942,   191473.        ]),
 array([ 2061219.55899115, -1489749.3300563 ,  1841053.93003564,
         -287797.499992  ,   -81079.01394892,   243417.        ]),
 array([ 2047713.44498159, -1360180.17003455,  1739391.33005863,
         -226420.59999183,   -80136.07178945,   253555.        ]),
 array([ 2009539.39298381, -1416394.85005213,  1769030.36007114,
         -280931.69998831,   -81102.31230948,   228800.        ]),
 array([ 1944952.22598135, -1849668.45006975,  1883146.68005439,
         -285094.19999497,   -80257.83340868,   236207.        ]),
 array([ 2122866.92897787, -1533106.42006614,  1877690.9600713 ,
         -296382.49999435,   -82754.275769  ,   257613.        ]),
 array([ 1985263.04199866, -1375389.07004903,  1778656.9600699 ,
         -285086.39999348,   -79784.25563902,   244951.        ]),
 array([ 2049833.86696853, -1561224.00004747,  1808875.69007162,
         -2

In [133]:
np.mean(errors)

13.1

In [125]:
w

array([151893.0080001 , -89538.88000133, 169260.5800028 , -26270.40000008,
        -4187.06775   ,  19006.        ])